# Creating XYZ-format files with ASE
We want to create an atom cluster structure with a crystallographic information file (CIF) as the input. CIF files have periodic boundaries, that we also want to remove. The atomic simulation environment (ase) can do this all in Python: [ASE Python library](https://wiki.fysik.dtu.dk/ase/)

In [1]:
from ase import io
from ase import Atoms
from ase.visualize import view
import numpy as np

In [2]:
stru = io.read('Structure_Models/beta-Ga2O3_CollCode83645.cif')

In [3]:
cell_parameters = stru.cell.cellpar()

In [4]:
#print(stru.cell)
#print(len(stru.get_positions()))

print(f'Cell loaded with unit cell a = {cell_parameters[0]} Å, b = {cell_parameters[1]} Å, c = {cell_parameters[2]} Å\n'\
      f'and alpha = {cell_parameters[3]} \N{DEGREE SIGN}, beta = {cell_parameters[4]} \N{DEGREE SIGN}, delta = {cell_parameters[5]} \N{DEGREE SIGN}')
print(f'There are {len(stru.get_positions())} atoms in the cell.')

Cell loaded with unit cell a = 12.214 Å, b = 3.0371 Å, c = 5.798099999999999 Å
and alpha = 90.0 °, beta = 103.83 °, delta = 90.0 °
There are 20 atoms in the cell.


In [5]:
# repeat the unit cell 1 x 2 x 2
supercell = stru.repeat((1,2,2))

In [6]:
print(f'No. of atoms in supercell: {len(supercell.get_positions())}')

No. of atoms in supercell: 80


In [7]:
view(supercell, viewer='x3d')

In [8]:
# list of atoms is now repeated and not sorted by Z.
print(supercell.get_atomic_numbers())

[31 31 31 31 31 31 31 31  8  8  8  8  8  8  8  8  8  8  8  8 31 31 31 31
 31 31 31 31  8  8  8  8  8  8  8  8  8  8  8  8 31 31 31 31 31 31 31 31
  8  8  8  8  8  8  8  8  8  8  8  8 31 31 31 31 31 31 31 31  8  8  8  8
  8  8  8  8  8  8  8  8]


In [9]:
def sort_atoms(stru: Atoms, # ase Atoms object
               elements: list,
              ) -> Atoms:
    """
     take in an Atoms object with a list of elements
     and return as Atoms object sorted by elements
     
     from (https://gist.github.com/HojeChun/4d6f5903b4837cd1a3ecb4e14bbd1d40)
    """
    n_elements = len(elements)
    element_encoder = dict(zip(elements, range(n_elements)))
    symbols_encoded = np.array([element_encoder[s] for s in stru.symbols])
    sort_indices = np.argsort(symbols_encoded)

    return stru[sort_indices]

In [10]:
# sort by Ga, O
sorted_supercell = sort_atoms(supercell, ['Ga', 'O'])

In [11]:
print(sorted_supercell.get_atomic_numbers())

[31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31 31
 31 31 31 31 31 31 31 31  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8]


In [12]:
sorted_supercell.write('Structure_Models/GO_1x2x2.xyz', 
                       comment=str('Ga2O3'), # 2nd line in the XYZ is just the formula
                      )

In [13]:
ga = list(supercell.get_atomic_numbers()).count(31)
o = list(supercell.get_atomic_numbers()).count(8)
print(f'There are {ga} Ga and {o} O atoms in the supercell.')

There are 32 Ga and 48 O atoms in the supercell.


This cell contains "dangling Ga" atoms that doesn't have finalized polyhedrons. Instead, an XYZ can be generated in the [VESTA software](https://jp-minerals.org/vesta/en/download.html).